In [1]:
import simpy
import random
import pandas as pd
import numpy as np
import math

In [21]:
class BikeShareSystem:
    def __init__(self, env, Full_Error, Empty_Error, num_stations, num_docks_per_station, destination_id, num_steps):
        self.env = env
        self.Full_Error = Full_Error
        self.Empty_Error = Empty_Error
        self.stations = [simpy.Container(env, capacity=num_docks_per_station) for i in range(num_stations)]
        self.destination_id = destination_id
        self.current_step = 8
        self.num_steps = num_steps
    def rent_bike(self, station_id):
        # Request a bike from the station
        yield self.stations[station_id].get(1)

    def return_bike(self, station_id):
        # Return a bike to the station
        yield self.stations[station_id].put(1)

    # Define a customer that uses the bike share system
    def Destination(self, station_id):
        print("Destination")
        U = np.random.uniform(0, 1)
        D = 1 if U > 1/2 else 0
        destination_id = D
        self.destination_id = destination_id

    def Departure(self, station_id):
        print("Departure")
        yield self.env.timeout(5*np.random.lognormal(0.1, np.sqrt(0.066)))

    def Customer_Arrival(self, station_id):
        print("Customer_Arrival")
        # STATION EMPTY
        if (self.stations[station_id].level) <= 0:
            self.Empty_Error += 1
        else:
            yield self.rent_bike(station_id)
            yield self.Destination(station_id)
            yield self.Departure(station_id)
            yield self.Bike_Arrival(self.destination_id)
            
    def Bike_Arrival(self, destination_id):
        print("Bike Arrival")

        #yield self.env.timeout(np.random.exponential(5))
        # STATION FULL
        if self.stations[destination_id].capacity <= (self.stations[destination_id].level):
            self.Full_Error += 1

        if (self.stations[destination_id].level) < self.stations[destination_id].capacity:
            yield self.return_bike(destination_id)
            

    def Retrial(self, station_id):  # retrial function
        print("Retrial")
        #yield self.env.timeout(np.random.exponential(5))

        # STATION FULL
        if self.stations[station_id].capacity <= (self.stations[station_id].level):
            self.Full_Error += 1
            yield self.Retrial(station_id)
        else:
            yield self.Arrival(station_id)
            
    def customer(self, id, station_id):
        # # # generate a Poisson interarrival time 
        interarrival_time = np.random.exponential(15)

        # # wait for the next customer to arrive
        yield self.env.timeout(interarrival_time)
        
        station_id = random.randint(0, len(bike_share.stations)-1)
        # # Randomly select a station to rent a bike from
        #print(f"Customer {id} wants to rent a bike from station {station_id}.")

        # Try to rent a bike from the selected station
        yield env.process(bike_share.Customer_Arrival(station_id))
        print(f"Customer {id} has rented a bike from station {station_id}.")

        # Return Station
        #print(f"Customer {id} wants to return the bike to station {self.destination_id}.")

        # Try to return the bike to the selected station
        yield env.process(bike_share.Departure(self.destination_id))
        print(f"Customer {id} has RETURNED the bike to station {self.destination_id}")
        
    def run(self):
        while self.current_step < self.num_steps:
            # Execute some action at each time step
            print("Step:", self.current_step)
            customer_count = 0
            for i in range(10):
                customer_count += 1
                env.process(self.customer(customer_count, bike_share))
                # Increment the current step and wait for the next time step
            self.current_step += 0.5
            yield self.env.timeout(1)

env = simpy.Environment()
bike_share = BikeShareSystem(env, 0, 0, 2, 5, 2, 12)

# Run the simulation
env.process(bike_share.run())
env.run()
print("Full Error Count:", bike_share.Full_Error)
print("Empty Error Count:", bike_share.Empty_Error)


Step: 8
Step: 8.5
Customer_Arrival
Customer 1 has rented a bike from station 1.
Departure
Step: 9.0
Customer_Arrival
Customer 10 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 7 has rented a bike from station 1.
Departure
Step: 9.5
Customer_Arrival
Customer 8 has rented a bike from station 1.
Departure
Step: 10.0
Customer_Arrival
Customer 6 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 6 has rented a bike from station 0.
Departure
Customer_Arrival
Customer 2 has rented a bike from station 0.
Departure
Step: 10.5
Customer_Arrival
Customer 2 has rented a bike from station 1.
Departure
Customer 7 has RETURNED the bike to station 2
Customer_Arrival
Customer 2 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 3 has rented a bike from station 1.
Departure
Customer_Arrival
Customer 7 has rented a bike from station 0.
Departure
Customer_Arrival
Customer 6 has rented a bike from station 0.
Departure
Customer_Arrival
Customer 5 h